In [69]:
import os 
import glob
import pandas as pd
import rasterio
import numpy as np
from skimage.color import label2rgb
import matplotlib.pyplot as plt

%matplotlib inline

In [1]:
def is_urban(val) :
    return (val==1 or val==2 or val==3)

In [83]:
def compute_urban_diff(prev, cur, transform, basedir) :
    
    lats = []
    lons = []
    for r in range(12) :
        for c in range(13) :
            prev_labels = np.load(os.path.join(basedir, '{}_{}_{}.npy'.format(prev, r, c))) 
            cur_labels = np.load(os.path.join(basedir, '{}_{}_{}.npy'.format(cur, r, c)))
            for r_ix in range(prev_labels.shape[0]) :
                for c_ix in range(prev_labels.shape[1]) :
                    if not is_urban(prev_labels[r_ix,c_ix]) and is_urban(cur_labels[r_ix,c_ix]) :
                        lon, lat = transform * (r*224 + r_ix, c*224 + c_ix)
                        lons.append(lon)
                        lats.append(lat)
    
    return pd.DataFrame({'lat':lats, 'lon':lons})

In [87]:
# groups = {
#     'austin': ['austin13', 'austin14', 'austin15'],
#     'dc': ['dc14', 'dc15'],
#     'denver': ['denver13', 'denver14', 'denver15'],
#     'sp': ['sp14', 'sp15']
# }
groups = {
    'austin': ['austin14', 'austin15']
}

In [89]:
for city, cityears in groups.iteritems() :
    tif_file = glob.glob('./cities/{}/*.tif'.format(city + '15'))[0]
    with rasterio.open(tif_file) as f :
        transform = f.transform
    pairs = zip(cityears[:-1], cityears[1:])
    base_dir = 'sp_outputs' if city=='sp' else 'kabul,austin,denver,dc_outputs'
    for pair in pairs :
        print(pair)
        df = compute_urban_diff(pair[0], pair[1], transform, base_dir)
        print(len(df))
        if len(df) < 500000 :
            df.to_csv(os.path.join('change_maps', pair[1] + '.csv'))
        else :
            df.sample(500000).to_csv(os.path.join('change_maps', pair[1] + '.csv'))

('austin14', 'austin15')
470610
